In [1]:
import numpy as np
import pandas as pd
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [2]:
import preprocessingWithMissingvalues

car_df= preprocessingWithMissingvalues.preprocess('./cardata.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15420 entries, 0 to 15419
Data columns (total 33 columns):
Month                   15420 non-null object
WeekOfMonth             15420 non-null int64
DayOfWeek               15420 non-null object
Make                    15420 non-null object
AccidentArea            15420 non-null object
DayOfWeekClaimed        15420 non-null object
MonthClaimed            15420 non-null object
WeekOfMonthClaimed      15420 non-null int64
Sex                     15420 non-null object
MaritalStatus           15420 non-null object
Age                     15420 non-null int64
Fault                   15420 non-null object
PolicyType              15420 non-null object
VehicleCategory         15420 non-null object
VehiclePrice            15420 non-null object
FraudFound              15420 non-null object
PolicyNumber            15420 non-null int64
RepNumber               15420 non-null int64
Deductible              15420 non-null int64
DriverRating           

/usr/local/anaconda35/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [3]:

car_lable= car_df['Lable']    #get the lable of the dataset
car_lable.shape
car_df.drop(['Lable'],inplace=True,axis=1)  #drop the lable feature from the dataframe.
car_df.head()

,Make_Accura,Make_BMW,Make_Chevrolet,Make_Dodge,Make_Ferrari,Make_Ford,Make_Honda,Make_Jaguar,Make_Lexus,Make_Mazda,...,AddressChange-Claim_under 6 months,NumberOfCars_1 vehicle,NumberOfCars_2 vehicles,NumberOfCars_3 to 4,NumberOfCars_5 to 8,NumberOfCars_more than 8,RepNo,Deductible,DriverRating,DaysDiff
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0.733333,0.00,0.000000,0.071795
1,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0.933333,0.25,1.000000,0.069231
2,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0.400000,0.25,0.666667,0.089744
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0.200000,0.25,0.333333,0.107692
4,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0.133333,0.25,0.000000,0.094872


In [4]:
#Divide the data into fraudulent and non-fraudulent data
columnlable= list(car_df.columns.values)

#create the dataframe for fraudulent and non-fraudulent data
nonFraudulent= pd.DataFrame(columns=columnlable)
nonFraudulentLable= pd.DataFrame(columns=['Lable'])

fraudulent= pd.DataFrame(columns=columnlable)
fraudulentLable= pd.DataFrame(columns=['Lable'])


j= 0
k= 0
for i in range(car_df.shape[0]):
    if(car_lable[i]==0):
        nonFraudulent.loc[j]= car_df.loc[i]
        nonFraudulentLable.loc[j]= 0.0
        j += 1
    else:
        fraudulent.loc[k]= car_df.loc[i]
        fraudulentLable.loc[i]= 1.0
        k += 1
    

In [5]:
print(nonFraudulent.shape)
print(fraudulent.shape)
nonFraudulent.head()

(14496, 97)
(923, 97)


,Make_Accura,Make_BMW,Make_Chevrolet,Make_Dodge,Make_Ferrari,Make_Ford,Make_Honda,Make_Jaguar,Make_Lexus,Make_Mazda,...,AddressChange-Claim_under 6 months,NumberOfCars_1 vehicle,NumberOfCars_2 vehicles,NumberOfCars_3 to 4,NumberOfCars_5 to 8,NumberOfCars_more than 8,RepNo,Deductible,DriverRating,DaysDiff
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.733333,0.00,0.000000,0.071795
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.933333,0.25,1.000000,0.069231
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.400000,0.25,0.666667,0.089744
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.200000,0.25,0.333333,0.107692
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.133333,0.25,0.000000,0.094872


In [6]:
'''
col= list(car_copy.columns.values)
nonFraudulent.columns= col
fraudulent.columns= col
'''

'\ncol= list(car_copy.columns.values)\nnonFraudulent.columns= col\nfraudulent.columns= col\n'

In [7]:
'''
nonFraudulent.head()
fraudulentLable.head()
'''

'\nnonFraudulent.head()\nfraudulentLable.head()\n'

In [8]:
#Note that for the train and test split, the lables and data have to be in numpy
nonFraudulentCarfeatures= nonFraudulent.values
nonFraudulentLable= nonFraudulentLable.values

fraudulentCarfeatures= fraudulent.values
fraudulentLable= fraudulentLable.values

print('non-Fraud: %d' %(nonFraudulentCarfeatures.shape[0]))
print('Fraud: %d' %(fraudulentCarfeatures.shape[0]))


non-Fraud: 14496
Fraud: 923


In [9]:
#create the class Dataset which returns the data and labels
from torch.utils.data.dataset import Dataset, TensorDataset
class myDataset(Dataset):
    
    def __init__(self,features,lables,transform= None):
        dataTensor= []
        lableTensor= []
        dataSize= features.shape[0]
        
        for data in range(dataSize):
            feature= features[data,:]
            #feature= torch.from_numpy(feature).float()
            feature= torch.Tensor(feature)
            dataTensor.append(feature)
            
            lable= np.asanyarray(lables[data])
            lable= torch.from_numpy(lable).float()
           
            #lable= torch.Tensor(lable)
            #print("lable:",lable)
            #assert(False)
           
            lableTensor.append(lable)
        
        #put everything in features and lables
        self.features= dataTensor
        self.lables= lableTensor
    
    
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self,index):
        
        feature= self.features[index]
        lable= self.lables[index]
        #print("get_item feature:",feature)
        #print("get_item lable:",lable)
        return feature,lable

In [10]:
#split the dataset into two part train and test
"""
X_train,X_test,y_train,y_test = train_test_split(carArray,labels,random_state=3,test_size=0.25)
print('xtrain:',type(X_train))
print(X_train.shape)
"""

"\nX_train,X_test,y_train,y_test = train_test_split(carArray,labels,random_state=3,test_size=0.25)\nprint('xtrain:',type(X_train))\nprint(X_train.shape)\n"

In [11]:
#make the dataset for train and test loader
mytrainDataset= myDataset(nonFraudulentCarfeatures,nonFraudulentLable)
mytestDataset= myDataset(fraudulentCarfeatures,fraudulentLable)



In [12]:
feature,lable= mytrainDataset.__getitem__(0)
print("feature:",feature)
print("lable:",lable)
print(feature.size())

feature: tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 1.0000, 1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000,
        1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000,
        0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000,
        0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 1.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        1.0000, 0.0000, 0.0000, 0.7333, 0.0000, 0.0000, 0.0718])
lable: tensor([0.])
torch.Size([97])


In [13]:

trainLoader1= torch.utils.data.DataLoader(mytrainDataset,batch_size=1,shuffle=True,num_workers=0)
testLoader1= torch.utils.data.DataLoader(mytestDataset,batch_size=1,shuffle=True,num_workers=0)

trainLoader2= torch.utils.data.DataLoader(mytestDataset,batch_size=1,shuffle=True,num_workers=0)
testLoader2= torch.utils.data.DataLoader(mytrainDataset,batch_size=1,shuffle=True,num_workers=0)




In [14]:
#Network architecture for the base autoencoders
class autoencoder1(nn.Module):
    def __init__(self):
        super(autoencoder1,self).__init__()
        self.encoder= nn.Sequential(
            nn.Linear(97,50),
            nn.ReLU(True),
            nn.Linear(50,25),
            nn.ReLU(True),nn.Linear(25,10),nn.ReLU(True),nn.Linear(10,5))
        self.decoder= nn.Sequential(
            nn.Linear(5,10),
            nn.ReLU(True),
            nn.Linear(10,25),
            nn.ReLU(True),
            nn.Linear(25,50),
            nn.ReLU(True),nn.Linear(50,97))
    
    def forward(self,x):
        x= self.encoder(x)
        x= self.decoder(x)
        return x;

In [15]:
#Network architecture for the base autoencoders
class autoencoder2(nn.Module):
    def __init__(self):
        super(autoencoder2,self).__init__()
        self.encoder= nn.Sequential(
            nn.Linear(97,75),
            nn.LeakyReLU(True),
            nn.Linear(75,25),
            nn.LeakyReLU(True),nn.Linear(25,10))
        self.decoder= nn.Sequential(
            nn.Linear(10,25),
            nn.LeakyReLU(True),
            nn.Linear(25,75),
            nn.LeakyReLU(True),
            nn.Linear(75,97))
    
    def forward(self,x):
        x= self.encoder(x)
        x= self.decoder(x)
        return x;

In [16]:
model1= autoencoder1()
criterion1= nn.MSELoss()
optimizer1= torch.optim.SGD(model1.parameters(), lr=0.001,momentum=0.9)

In [17]:
model2= autoencoder2()
criterion2= nn.MSELoss()
optimizer2= torch.optim.SGD(model2.parameters(), lr=0.001,momentum=0.9)

In [18]:
#train the model1
for epoch in range(10):
    running_loss1= 0.0
    for i, (feature,lable) in enumerate(trainLoader1):
        
        #gets the inputs
        inputs1= torch.tensor(feature)
        lables1= torch.tensor(lable)
        lables1= lables1.type(torch.LongTensor)
       
        #print(inputs.size())
        #print(inputs)
        
        # =====================forward====================
        output1 = model1(inputs1)
        #print(output)
        #loss = criterion(output,inputs).mean(dim=1)
        loss1 = criterion1(output1,inputs1)
        '''
        print("loss:",loss)
        print("output:",output)
        print("input:",inputs)
        '''    
        
         # ===================backward====================
        optimizer1.zero_grad()
        loss1.backward()
        optimizer1.step()
        
        # =======print the statistics
        running_loss1 += loss1.item()
        
        #print("i: ",i)
    #if i%100 == 0:              #print every 2000 mini-batches
    
    print('[%d] loss: %.3f' %
              (epoch + 1,  running_loss1 / nonFraudulentCarfeatures.shape[0]))
      #running_loss = 0.0

        

[1] loss: 0.084
[2] loss: 0.072
[3] loss: 0.072
[4] loss: 0.072
[5] loss: 0.072
[6] loss: 0.072
[7] loss: 0.072
[8] loss: 0.072
[9] loss: 0.072
[10] loss: 0.072


In [19]:
#train the model2
for epoch in range(50):
    running_loss2= 0.0
    for i, (feature,lable) in enumerate(trainLoader2):
        
        #gets the inputs
        inputs2= torch.tensor(feature)
        lables2= torch.tensor(lable)
        lables2= lables2.type(torch.LongTensor)
       
        #print(inputs.size())
        #print(inputs)
        
        # =====================forward====================
        output2= model2(inputs2)
        
        #print(output)
        #loss = criterion(output,inputs).mean(dim=1)
        loss2= criterion2(output2,inputs2)
        #print("loss2: ",loss2)
        '''
        print("loss:",loss)
        print("output:",output)
        print("input:",inputs)
        '''    
        
         # ===================backward====================
        optimizer2.zero_grad()
        loss2.backward()
        optimizer2.step()
        
        # =======print the statistics
        running_loss2 += loss2.item()
        
        #print("i: ",i)
    #if i%100 == 0:              #print every 2000 mini-batches
    
    print('[%d] loss: %.3f' %
              (epoch + 1,  running_loss2 / fraudulentCarfeatures.shape[0]))
      #running_loss = 0.0

[1] loss: 0.128
[2] loss: 0.070
[3] loss: 0.069
[4] loss: 0.068
[5] loss: 0.068
[6] loss: 0.068
[7] loss: 0.068
[8] loss: 0.068
[9] loss: 0.068
[10] loss: 0.068
[11] loss: 0.067
[12] loss: 0.067
[13] loss: 0.067
[14] loss: 0.067
[15] loss: 0.067
[16] loss: 0.066
[17] loss: 0.066
[18] loss: 0.066
[19] loss: 0.065
[20] loss: 0.065
[21] loss: 0.064
[22] loss: 0.063
[23] loss: 0.063
[24] loss: 0.062
[25] loss: 0.062
[26] loss: 0.061
[27] loss: 0.061
[28] loss: 0.061
[29] loss: 0.060
[30] loss: 0.060
[31] loss: 0.060
[32] loss: 0.059
[33] loss: 0.059
[34] loss: 0.059
[35] loss: 0.058
[36] loss: 0.058
[37] loss: 0.057
[38] loss: 0.057
[39] loss: 0.056
[40] loss: 0.056
[41] loss: 0.055
[42] loss: 0.055
[43] loss: 0.055
[44] loss: 0.054
[45] loss: 0.054
[46] loss: 0.053
[47] loss: 0.053
[48] loss: 0.052
[49] loss: 0.052
[50] loss: 0.052


In [22]:

#test the model
def modelTest(Loader):
    predictedLable= []
    for i,(feature,lable) in enumerate(Loader):
        inputs= torch.tensor(feature)
        
        #test the model 1
        output1= model1(inputs)
        loss3= criterion1(output1,inputs)
        
        
        #test the model 2
        #output2= model2(inputs)
        #loss4= criterion2(output2,inputs)
        
        print("inputs: ",inputs)
        print("output1: ",output1)
        print("output2:",output2)
        print("[loss1 %d]" %(loss3))
        print("inputs: ",inputs)
        print("[loss2 %d]" %(loss4))
        assert(False)
        
        #print(loss)
        '''
        if(loss>0.05):
            predictedLable.append(0)
        else:
            predictedLable.append(1)
        '''
        
        if(loss1<loss2):
            predictedLable.append(0)
        else:
            predictedLable.append(1)
    return predictedLable
     

In [23]:


predictedLable= modelTest(testLoader1)
ones= predictedLable.count(1)
zeros= predictedLable.count(0)
#print('%d %d' %(plus,minus))


inputs:  tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000, 1.0000, 0.0000,
         1.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 1.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000,
         1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000,
         1.0000, 0.0000, 0.0000, 0.4000, 0.2500, 0.6667, 0.0718]])
output1:  tensor([[ 0.0325,  0.0021,  0.1211,  0.0108, -0.0001,  0.0366,  0.1730, -0.0005,
          0.0005,  0.14

NameError: name 'loss4' is not defined

In [ ]:
print("%d %d"%(ones,zeros))

In [ ]:
predictedLable= modelTest(testLoader2)
ones= predictedLable.count(1)
zeros= predictedLable.count(0)


In [ ]:
print("%d %d"%(ones,zeros))

In [ ]:
#save the model
#torch.save(model.state_dict(),'./model_path')